<h1><u>RNN thesis</u></h1>
<h4>Omer Nivron</h4>
15098443

<h3>Package loading</h3>

In [2]:
import numpy as np
import sys as sys
import random as rd
import tensorflow as tf
import gzip
from __future__ import division
from tensorflow.python.ops import array_ops, control_flow_ops
import pandas as pd
import os
import math
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time
import csv


In [ ]:
import numpy as np
import boto3
import tensorflow as tf
import pandas as pd
import os
import cStringIO
! sudo pip install s3fs
import s3fs
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Data creation

In [4]:
def np_pad_tr_x(x_tr, batch_size, str_idx, zero_array_x, length_vec):
    start = 0
    for i in range(batch_size):
        if i > 0:
            end = end + length_vec[i] 
        else:
            end = length_vec[0]
        zero_array_x[ int(str_idx[i]) : int(str_idx[i] + length_vec[i]) ] = x_tr[ start : end ] 
        start = end

    return(zero_array_x)

In [5]:
def range_bet_col_t_col_n_append(col_1 ,col_2):
    app_ranges=[]
    for i in range(col_1.shape[0]):
        single_range = range((col_1[i]).astype(int), (col_2[i]).astype(int))
        app_ranges = np.append(app_ranges, single_range)
    return(app_ranges)

In [6]:
def algeb_geom_series(mode ,start ,jump ,length):
    u = np.empty((length,))
    u[0] = start
    u[1:] = jump
    if (mode == 0):
        series=np.cumsum(u)
    if (mode == 1):
        series=np.cumprod(u)
    return(series)

In [7]:
def sort_pd_df_by_ext_vec(df,ext_sor_vec, cols):
    df_s = df[((df[cols[0]]).astype(int)).isin(ext_sor_vec)] #
    df_s['sort_cat'] = pd.Categorical(df_s[cols[0]],categories = ext_sor_vec,ordered = True)
    if len(cols) > 1:
        df_s.sort_values(['sort_cat',cols[1]] ,inplace = True)
    
    else:
        df_s.sort_values(['sort_cat'],inplace = True) 
    
    df_s.reset_index(inplace = True)
    df_ = df_s.drop(['sort_cat','index'] ,axis = 1)
    
    return(df_)

In [8]:
def rnn_model(model ,n_hidden ,layers):
    if (layers == 1):
        if(model == 'lstm'):
            cell = tf.nn.rnn_cell.LSTMCell(n_hidden ,state_is_tuple=True)
        else:
            cell = tf.nn.rnn_cell.GRUCell(n_hidden)   
    else:
        if(model == 'lstm'):
            lstm = tf.nn.rnn_cell.LSTMCell(n_hidden ,state_is_tuple=True)
            cell = tf.nn.rnn_cell.MultiRNNCell([lstm]*layers)

        else:
            gru = tf.nn.rnn_cell.GRUCell(n_hidden)
            cell = tf.nn.rnn_cell.MultiRNNCell([gru] * layers)
    return(cell)

############################################


Loading Fake Data

In [8]:
def trainSamples(viewers,videos,probab,viewerFeat,videoFeat,contxFeat):
    trData = {} # trData = Dictionary with training data. This is histrory of viewer and video iteraction
    X = {} # X  = Dictionary with viewer features as arrays
    Y ={} # Y  = Dictionary with video features as arrays
    for i in range(viewers):
        X[i] = np.random.rand(viewerFeat)
        a = 0 # timing of the video for a particular user,... 
                #to give the order in which the videos have been watched
        for j in range(videos):
            if int(np.random.binomial(1,probab ,1)[0]):
                trData[(i,j,a)] = np.random.rand(contxFeat)
                a+=1 # when a video is watched, we increase the value of a by 1 
            if i==0:
                Y[j] = np.random.rand(videoFeat)
    return X,Y,trData

In [9]:
viewers = 1000  #number of viewers
videos =1000  #number of videos
probab = 0.05  #probability of a viewer watching any one video
viewerFeat = 310  #number of features describing a veiwer
videoFeat = 300   #number of features describing a video
contxFeat = 15 # number of contextual features
# X  = Dictionary with viewer features as arrays
# Y  = Dictionary with video features as arrays
# trData = Dictionary with training data. This is histrory of viewer and video iteraction
X ,Y ,trData = trainSamples(viewers ,videos ,probab ,viewerFeat ,videoFeat ,contxFeat)

### transform a dict to numpy array

In [10]:
user_feat_inp = np.array([X[key] for key in sorted(X.keys())]) 
vid_feat_inp = np.array([Y[key] for key in sorted(Y.keys())])

### Add a column key specifying user

In [11]:
key_user = np.asarray(range(user_feat_inp.shape[0])).reshape(user_feat_inp.shape[0] ,1)
key_vid = np.asarray(range(vid_feat_inp.shape[0])).reshape(vid_feat_inp.shape[0] ,1)
user_feat_inp_w_key = np.concatenate((user_feat_inp ,key_user),axis = 1)
vid_feat_inp_w_key = np.concatenate((vid_feat_inp ,key_vid),axis = 1)
user_vid_time = trData.keys()

### transform numpy to pandas df in order to use easy merging

In [12]:
user_feat_inp_w_key_df = pd.DataFrame(user_feat_inp_w_key) 
vid_feat_inp_w_key_df = pd.DataFrame(vid_feat_inp_w_key)
user_vid_time_df = pd.DataFrame(user_vid_time)

## Construct true_y

### sort values by user and time watched

In [13]:
rr = user_vid_time_df.sort_values([0 ,2]).reset_index()

### get categorical column for each video with 1 where watched and 0 otherwise

In [14]:
h = pd.get_dummies(rr[1],prefix = 'vid_')
y_tr_p_w = pd.concat([rr.reset_index(drop = True), h], axis = 1)
y_tr_p_w.rename(columns = {0: 'user'}, inplace = True)
y_tr_p_w.rename(columns = {1: 'mov'}, inplace = True)
y_tr_p_w.rename(columns = {2: 'rank'}, inplace = True)
y_tr_p_w['desired'] = np.argmax(np.array(y_tr_p_w.iloc[:,4:]) ,1)

### We get for each user one row with 1's in the positions watched incl. user column(0) 

In [15]:
max_watch = (y_tr_p_w.groupby('user',axis = 0).sum().iloc[:
                                ,int(np.array(np.where(y_tr_p_w.columns=='rank'))):-1].sum(axis=1)).reset_index()

### Merge two pandas df 

In [16]:
vid_feat_inp_w_key_df.rename(columns = {300: 'mov_id'}, inplace = True)
user_feat_inp_w_key_df.rename(columns = {310: 'user_id'}, inplace = True)
user_vid_time_df.rename(columns = {0: 'user_id'}, inplace = True)
user_vid_time_df.rename(columns = {1: 'mov_id'}, inplace = True)
user_w_vid_tim_and_feat = user_vid_time_df.merge(user_feat_inp_w_key_df 
                                                 ,how = 'inner',on = 'user_id', sort = False)
user_vid_time_vidfeat_usefit = user_w_vid_tim_and_feat.merge(vid_feat_inp_w_key_df 
                                                             ,how = 'inner' ,on='mov_id' ,sort = False)
user_vid_time_vidfeat_usefit.rename(columns={'2_x': 'time_watch'} ,inplace=True)
user_vid_time_vidfeat_usefit_sorted = user_vid_time_vidfeat_usefit.sort_values(['user_id', 'time_watch']) 


In [ ]:
user_vid_time_vidfeat_usefit_sorted_dis = user_vid_time_vidfeat_usefit_sorted

### Split training and testing data

In [17]:
tr_users = viewers * 0.8
tr_y = y_tr_p_w[y_tr_p_w['user'] < tr_users]
te_y = y_tr_p_w[~y_tr_p_w['user'].isin(tr_y['user'])]

################################################

##################################################

<h2>Configurations</h2>

In [354]:
tf.reset_default_graph()

<h3>LSTM with X units</h3>

In [355]:
n_opt_epoch = 1000
model = 'lstm'
layers = 1
n_samples = 2000
top_k = 15
top_l = 10
top_r = 5
fixed = 10
top_all = 2000
n_users = num_users
te_users = int(np.ceil(num_users*0.2))
n_feature = num_user_feat + num_video_feat + contex_feat #623
n_feature_dis = num_video_feat
lr_rat = 0.001
num_video = h.shape[1]
beta = 0.01

if tr_users > 500:
    batch_size = 500
else:
    batch_size  = tr_users
        
n_hidden = 64 # hidden layer num of features
n_hidden_dis = 64 

Tensorboard saving paths

In [356]:
! mkdir -p ./reco_rnn/tensor_plot/dis_xent/
! mkdir -p ./reco_rnn/dis_xent/
#logs_path = "/Users/onivron/Desktop/reco_rnn/tensor_plot/p_w/"
logs_path = "./reco_rnn/tensor_plot/dis_xent/"

<h3>Var creation</h3>
We build data placeholders & variables 

In [357]:
with tf.name_scope('input'):
    dislike_inp = tf.placeholder("float32" ,[batch_size ,fixed, n_feature_dis] ,name = 'dislike_inp') 
    dynam_input = tf.placeholder("float32" ,[batch_size ,None, n_feature] ,name = 'dynam_input') 
    y_true = tf.placeholder("int32",[None, 1] ,name = 'Input_y')
    max_batch_length = tf.placeholder("float32" ,[batch_size] ,name = 'max_leng')
    tr_rw = tf.placeholder("int32" ,[None, 1] ,name = '_rw')
    tr_rw_n_desired_rep = tf.placeholder("int32" ,[None, 2] ,name = 'rw_rep')
    tr_rw_non_rep = tf.placeholder("int32" ,[None, 2] ,name = 'non_rep')
    
with tf.name_scope("weights"):
    W = {'out_1': tf.Variable(tf.random_normal([1500 ,num_video])  ,name = 'out_1'),
        'out_2': tf.Variable(tf.random_normal([n_hidden ,num_video]) ,name = 'w_2') , 
        'dis': tf.Variable(tf.random_normal([fixed * n_feature_dis, 2000])  ,name = 'dis'),
        'dis_2': tf.Variable(tf.random_normal([2000 ,1500])  ,name = 'dis_2')}
    
with tf.name_scope("biases"):
    b = {'out_1': tf.Variable(tf.random_normal([num_video]) ,name = 'b_1'),
        'out_2': tf.Variable(tf.random_normal([num_video]) ,name = 'b_2') , 
        'dis': tf.Variable(tf.random_normal([2000])  ,name = 'b_dis'),
        'dis_2': tf.Variable(tf.random_normal([1500])  ,name = 'b_dis_2')}

<h3>Define model</h3>

In [358]:
def RNN(dynam_input, dislike_inp 
        ,max_batch_length
         ,W ,b ,model):
    
    lstm_cell = rnn_model(model ,n_hidden ,layers)
    
    outputs ,states = tf.nn.dynamic_rnn(lstm_cell ,inputs = dynam_input
                                     ,dtype = tf.float32 ,sequence_length = max_batch_length)
    
    
    out_shaped = tf.reshape(outputs ,[-1 ,n_hidden])
    
    dislike_layer = (tf.matmul(tf.reshape(dislike_inp, [-1, fixed * n_feature_dis]) 
                                         , W['dis']) + b['dis']) 
    
    dislike_layer_embeded = tf.nn.relu(tf.matmul(dislike_layer ,W['dis_2']) + b['dis_2']) 
    
    dislike_score = tf.nn.relu(tf.matmul(dislike_layer_embeded, W['out_1']) + b['out_1']) 
    
    like_score = tf.reshape((tf.matmul(out_shaped ,W['out_2']) + b['out_2']), [-1, batch_size ,num_video])
    
    layer_3 = dislike_score + like_score
    
    relative_like = tf.divide(like_score, layer_3)
        
    return (relative_like, dislike_score, like_score, layer_3) 

<h3>Define loss, accuracy & optimizer</h3>

In [359]:
_relative_like, _dislike_score, _like_score, y_pred  = RNN(dynam_input, dislike_inp, max_batch_length, W, b, model)

with tf.name_scope('like_facotr'):
    like_factor = tf.gather_nd(params = tf.reshape(_relative_like, [-1, num_video]), indices = tr_rw_non_rep)

relevant_y_pred = tf.reshape(tf.gather(params = tf.reshape(y_pred
                                        ,[-1 ,num_video]), indices = tr_rw), [-1, num_video])

top_vals, top_indice = tf.nn.top_k(relevant_y_pred, n_samples)


with tf.name_scope('cross_entropy'):

    xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = relevant_y_pred, 
                            labels = tf.reshape(y_true, [-1]))) 
                            
                            
                            
    regularizer_1 = tf.nn.l2_loss(tf.abs(W['out_2']))
    regularizer_2 = tf.nn.l2_loss(tf.abs(W['out_1'])) 
    regularizer_3 = tf.nn.l2_loss(tf.abs(W['dis_2'])) 
    regularizer_4 =  tf.nn.l2_loss(tf.abs(W['dis']))
    
    
    xent_regu = tf.reduce_mean(xent + beta * regularizer_1 
                               + beta * regularizer_2
                               + beta * regularizer_3
                               + beta * regularizer_4
                              )
    
    
with tf.name_scope('Accuracy_top_r'):
    top_15_indx_r = tf.slice(top_indice,[0,0],[-1, top_r])
    to_bool_r = tf.reduce_sum(tf.cast(tf.equal(y_true, top_15_indx_r), tf.float32), 1)
    accuracy_r = tf.reduce_mean(to_bool_r)   
    
with tf.name_scope('Accuracy_top_l'):
    top_15_indx_l = tf.slice(top_indice,[0,0],[-1, top_l])
    to_bool_l = tf.reduce_sum(tf.cast(tf.equal(y_true, top_15_indx_l), tf.float32), 1)
    accuracy_l = tf.reduce_mean(to_bool_l)
    
with tf.name_scope('Accuracy_top_k'):
    top_15_indx_k = tf.slice(top_indice,[0,0],[-1,top_k])
    to_bool_k = tf.reduce_sum(tf.cast(tf.equal(y_true, top_15_indx_k), tf.float32), 1)
    accuracy_k = tf.reduce_mean(to_bool_k)

In [360]:
with tf.name_scope('train'):    
    optimizer = tf.train.AdamOptimizer(learning_rate = lr_rat).minimize(xent_regu)

<h3>Tensorboard set-up</h3>

In [361]:
tf.summary.scalar("cost", xent_regu)
tf.summary.scalar("Accuracy_top_r", accuracy_r)
tf.summary.scalar("Accuracy_top_k", accuracy_k)
tf.summary.scalar("Accuracy_top_l", accuracy_l)

summary_op = tf.summary.merge_all()
writer_opt = tf.summary.FileWriter(logs_path ,graph = tf.get_default_graph())
saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)

<h2>Training - optimization</h2>

In [362]:
init = tf.global_variables_initializer()
sess = tf.Session() 
sess.run(init)

In [364]:
save_MDir = './reco_rnn/dis_xent/'
save_model = os.path.join(save_MDir,'accu')
sess=tf.Session() 
sess.run(tf.global_variables_initializer())
saver.restore(sess = sess,save_path = save_model)

INFO:tensorflow:Restoring parameters from ./reco_rnn/dis_xent/accu


In [ ]:
with open('dis_xent.csv', 'wb') as csvfile:
    wr = csv.writer(csvfile, delimiter='\t', lineterminator='\n')
    for epoch in range(int(n_opt_epoch)):
        if(epoch != 0):
            writer_opt.add_summary(summary ,epoch)
        for batch_n in range(int(n_users / batch_size)):
            rw_to_chose = np.random.choice((tr_likes["user_id"].unique()) ,batch_size ,replace = False)
            x_tr = sort_pd_df_by_ext_vec(tr_likes
                                         , rw_to_chose, cols = ['user_id','rank'])
            x_tr_dis = sort_pd_df_by_ext_vec(train_dislike
                                         , rw_to_chose, cols = ['user_id','rank'])
            y_batch = sort_pd_df_by_ext_vec(tr_y ,rw_to_chose,cols = ['user_id','rank'])
            length_vec = sort_pd_df_by_ext_vec(df = max_watch ,ext_sor_vec = rw_to_chose ,cols = ['user_id']) 
            length_max = max(length_vec.iloc[:,1])
            str_idx = algeb_geom_series(0 ,start = 0 ,jump = length_max ,length = batch_size)
            end_idx = np.append(length_vec.iloc[0, 1], length_vec.iloc[1:, 1:] + str_idx[1:].reshape(batch_size-1, -1))
            app_range = range_bet_col_t_col_n_append(str_idx, end_idx)
            trial_size = (int(batch_size * length_max), n_feature)
            trial_size_dis = (int(batch_size * fixed), n_feature_dis)
            zero_array_x = np.zeros(trial_size) 
            zero_array_x_dis = np.zeros(trial_size_dis)
            _x_tr = np_pad_tr_x(x_tr.iloc[:,3:], batch_size, str_idx.astype(int),
                                            zero_array_x, length_vec.iloc[:,1].astype(int))

            rep_ind = np.concatenate((np.repeat(app_range, n_samples).reshape(-1, 1)
                                      , np.repeat(y_batch['desired'], n_samples).reshape(-1, 1)), axis = 1)

            (acc_r, acc_l ,acc_k, xen, _, summary) = sess.run([accuracy_r, accuracy_l
                                                                                  , accuracy_k, xent_regu 
                                                                                  ,optimizer, summary_op]
                                                      ,feed_dict = {dynam_input: _x_tr.reshape(batch_size, -1, n_feature)
                                                      ,dislike_inp: np.array(x_tr_dis.iloc[:,3:]).reshape(batch_size, -1, n_feature_dis)
                                                      ,tr_rw_n_desired_rep: rep_ind
                                                      ,tr_rw : app_range.reshape(-1, 1) 
                                                      ,y_true: y_batch['desired'].reshape(-1, 1)
                                                      ,max_batch_length: length_vec.iloc[:, 1]})
            wr.writerow([acc_r])
            wr.writerow([acc_l])
            wr.writerow([acc_k])
            #wr.writerow([tr_mrr])
            if (epoch%10 == 0):
                print('epoch number:', epoch, 'loss:', xen, 'accuracy 15:', acc_k)
                print('epoch number:', epoch, 'accuracy 10:', acc_l)
                print('epoch number:', epoch, 'accuracy 5:', acc_r)


        folder = './reco_rnn/dis_xent/'
        save_path = saver.save(sess, folder + 'accu')

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/python2.7/dist-packages/ipy

epoch number: 0 loss: 132258.0 accuracy 15: 0.00194633
epoch number: 0 accuracy 10: 0.00133914
epoch number: 0 accuracy 5: 0.000615507
epoch number: 0 loss: 131497.0 accuracy 15: 0.00212514
epoch number: 0 accuracy 10: 0.00138093
epoch number: 0 accuracy 5: 0.000694599
epoch number: 0 loss: 131049.0 accuracy 15: 0.00195626
epoch number: 0 accuracy 10: 0.00139852
epoch number: 0 accuracy 5: 0.000707584
epoch number: 10 loss: 121605.0 accuracy 15: 0.00319317
epoch number: 10 accuracy 10: 0.00217566
epoch number: 10 accuracy 5: 0.00117469
epoch number: 10 loss: 121370.0 accuracy 15: 0.00291686
epoch number: 10 accuracy 10: 0.00205847
epoch number: 10 accuracy 5: 0.00115008
epoch number: 10 loss: 121085.0 accuracy 15: 0.00320202
epoch number: 10 accuracy 10: 0.00212362
epoch number: 10 accuracy 5: 0.00111988
epoch number: 20 loss: 116002.0 accuracy 15: 0.00352084
epoch number: 20 accuracy 10: 0.00226752
epoch number: 20 accuracy 5: 0.00125332
epoch number: 20 loss: 115829.0 accuracy 15: 0.

epoch number: 470 loss: 47903.8 accuracy 15: 0.204441
epoch number: 470 accuracy 10: 0.186763
epoch number: 470 accuracy 5: 0.15871
epoch number: 470 loss: 47872.8 accuracy 15: 0.196055
epoch number: 470 accuracy 10: 0.178692
epoch number: 470 accuracy 5: 0.151732
epoch number: 480 loss: 47005.5 accuracy 15: 0.215866
epoch number: 480 accuracy 10: 0.196923
epoch number: 480 accuracy 5: 0.168508
epoch number: 480 loss: 46974.8 accuracy 15: 0.207636
epoch number: 480 accuracy 10: 0.189898
epoch number: 480 accuracy 5: 0.162328
epoch number: 480 loss: 46944.0 accuracy 15: 0.199036
epoch number: 480 accuracy 10: 0.18155
epoch number: 480 accuracy 5: 0.154394
epoch number: 490 loss: 46086.8 accuracy 15: 0.210004
epoch number: 490 accuracy 10: 0.191815
epoch number: 490 accuracy 5: 0.163634
epoch number: 490 loss: 46056.3 accuracy 15: 0.203843
epoch number: 490 accuracy 10: 0.185326
epoch number: 490 accuracy 5: 0.157634
epoch number: 490 loss: 46025.9 accuracy 15: 0.204593
epoch number: 490

In [368]:
6/7

0.8571428571428571

In [370]:
acc_k


0.20332171

# testing

In [ ]:
rw_to_chose = np.random.choice(range(int(tr_users), n_users)
                               , n_users - int(tr_users), replace = False)
x_tr = sort_pd_df_by_ext_vec(user_vid_time_vidfeat_usefit_sorted
                             , rw_to_chose, cols = ['user_id','time_watch'])
x_tr_dis = sort_pd_df_by_ext_vec(user_vid_time_vidfeat_usefit_sorted_dis
                             , rw_to_chose, cols = ['user_id','time_watch'])
y_batch = sort_pd_df_by_ext_vec(te_y ,rw_to_chose,cols = ['user','rank'])
length_vec = sort_pd_df_by_ext_vec(df = max_watch ,ext_sor_vec = rw_to_chose ,cols = ['user']) 
length_max = max(length_vec.iloc[:,1])
str_idx = algeb_geom_series(0 ,start = 0 ,jump = length_max ,length = batch_size)
end_idx = np.append(length_vec.iloc[0, 1], length_vec.iloc[1:, 1:] + str_idx[1:].reshape(batch_size-1, -1))
app_range = range_bet_col_t_col_n_append(str_idx, end_idx)
trial_size = (int(batch_size * length_max), n_feature)
zero_array_x = np.zeros(trial_size) ; zero_array_x_dis = np.zeros(trial_size)
_x_tr = np_pad_tr_x(x_tr.iloc[:,3:], batch_size, str_idx.astype(int),
                                zero_array_x, length_vec.iloc[:,1].astype(int))
_x_tr_dis = np_pad_tr_x(x_tr_dis.iloc[:,3:], batch_size, str_idx.astype(int),
                                zero_array_x_dis, length_vec.iloc[:,1].astype(int))  
rep_ind = np.concatenate((np.repeat(app_range, n_samples).reshape(-1, 1)
                          , np.repeat(y_batch['desired'], n_samples).reshape(-1, 1)), axis = 1)
non_rep = np.concatenate((app_range).reshape(-1, 1)
                          , (y_batch['desired']).reshape(-1, 1), axis = 1)
correct_inco, lik_fac, rel_lik, dis, lik, test_accuracy_r, test_accuracy_l ,test_accuracy_k, test_mrr, test_user_acc, 
test_loss  = sess.run([to_bool_k, like_factor, _relative_like, _dislike_score, _like_score, accuracy_r, accuracy_l , accuracy_k, mrr, to_bool_user, p_wise]
                                          ,feed_dict = {dynam_input: _x_tr.reshape(batch_size, -1, n_feature)
                                          ,dislike_inp: _x_tr_dis.reshape(batch_size, -1, n_feature)
                                          ,tr_rw_n_desired_rep: rep_ind
                                          ,tr_rw_non_rep: non_rep  
                                          ,tr_rw : app_range.reshape(-1, 1) 
                                          ,y_true: y_batch['desired'].reshape(-1, 1)
                                          ,max_batch_length: length_vec.iloc[:, 1]})

print('accuracy top 5:', test_accuracy_r) 
print('accuracy top 10:', test_accuracy_l) 
print('accuracy top 15:', test_accuracy_k) 

In [ ]:
data = {}
for user in range(n_users): 
    user_like_fac = lik_fac[str_idx[user] : end_idx[user]].reshape(-1, 1)
    user_cor_inco = correct_inco[str_idx[user] : end_idx[user]].reshape(-1, 1)
    data{user} = user_like_fac * user_cor_inco

In [ ]:
from matplotlib import pyplot as plt
heatmap = plt.pcolor(data{}, cmap=matplotlib.cm.Blues)

<h3>End session</h3>

In [ ]:
sess.close()